In [1]:
import pandas as pd
import numpy as np

# Load the dataset you previously saved from the API
df = pd.read_csv("../data/f1_2023_results.csv")

df.head()

,number,position,positionText,points,grid,laps,status,Driver.driverId,Driver.permanentNumber,Driver.code,...,Time.millis,Time.time,FastestLap.rank,FastestLap.lap,FastestLap.Time.time,FastestLap.AverageSpeed.units,FastestLap.AverageSpeed.speed,raceName,round,season
0,1,1,1,25,1,57,Finished,max_verstappen,33,VER,...,5636736.0,1:33:56.736,6.0,44.0,1:36.236,kph,202.452,Bahrain Grand Prix,1,2023
1,11,2,2,18,2,57,Finished,perez,11,PER,...,5648723.0,+11.987,7.0,37.0,1:36.344,kph,202.225,Bahrain Grand Prix,1,2023
2,14,3,3,15,5,57,Finished,alonso,14,ALO,...,5675373.0,+38.637,5.0,36.0,1:36.156,kph,202.620,Bahrain Grand Prix,1,2023
3,55,4,4,12,4,57,Finished,sainz,55,SAI,...,5684788.0,+48.052,14.0,37.0,1:37.130,kph,200.588,Bahrain Grand Prix,1,2023
4,44,5,5,10,7,57,Finished,hamilton,44,HAM,...,5687713.0,+50.977,10.0,36.0,1:36.546,kph,201.802,Bahrain Grand Prix,1,2023


In [2]:
df.columns

Index(['number', 'position', 'positionText', 'points', 'grid', 'laps',
       'status', 'Driver.driverId', 'Driver.permanentNumber', 'Driver.code',
       'Driver.url', 'Driver.givenName', 'Driver.familyName',
       'Driver.dateOfBirth', 'Driver.nationality', 'Constructor.constructorId',
       'Constructor.url', 'Constructor.name', 'Constructor.nationality',
       'Time.millis', 'Time.time', 'FastestLap.rank', 'FastestLap.lap',
       'FastestLap.Time.time', 'FastestLap.AverageSpeed.units',
       'FastestLap.AverageSpeed.speed', 'raceName', 'round', 'season'],
      dtype='object')

In [3]:
# Ensure numeric types where needed
df['position'] = pd.to_numeric(df['position'], errors='coerce')
df['grid'] = pd.to_numeric(df['grid'], errors='coerce')
df['round'] = pd.to_numeric(df['round'], errors='coerce')
df['season'] = pd.to_numeric(df['season'], errors='coerce')

# Create podium flag if not already present
if 'podium' not in df.columns:
    df['podium'] = np.where(df['position'] <= 3, 1, 0)

df[['Driver.code', 'Constructor.name', 'grid', 'position', 'podium']].head()

,Driver.code,Constructor.name,grid,position,podium
0,VER,Red Bull,1,1,1
1,PER,Red Bull,2,2,1
2,ALO,Aston Martin,5,3,1
3,SAI,Ferrari,4,4,0
4,HAM,Mercedes,7,5,0


In [4]:
# Map finishing position to points (F1 standard scoring)
points_map = {
    1: 25,
    2: 18,
    3: 15,
    4: 12,
    5: 10,
    6: 8,
    7: 6,
    8: 4,
    9: 2,
    10: 1
}

df['points'] = df['position'].map(points_map).fillna(0)

df[['Constructor.name', 'position', 'points']].head()

,Constructor.name,position,points
0,Red Bull,1,25.0
1,Red Bull,2,18.0
2,Aston Martin,3,15.0
3,Ferrari,4,12.0
4,Mercedes,5,10.0


In [5]:
# Sort to ensure cumulative logic is correct
df = df.sort_values(by=['season', 'round', 'Constructor.name'])

# Cumulative constructor points up to (but not including) this race
df['constructor_cum_points'] = (
    df.groupby(['season', 'Constructor.name'])['points']
      .cumsum()
      .shift(1)
      .fillna(0)
)

df[['season', 'round', 'Constructor.name', 'points', 'constructor_cum_points']].head(10)

,season,round,Constructor.name,points,constructor_cum_points
7,2023,1,Alfa Romeo,4.0,0.0
15,2023,1,Alfa Romeo,0.0,4.0
10,2023,1,AlphaTauri,0.0,4.0
13,2023,1,AlphaTauri,0.0,0.0
8,2023,1,Alpine F1 Team,2.0,0.0
17,2023,1,Alpine F1 Team,0.0,2.0
2,2023,1,Aston Martin,15.0,2.0
5,2023,1,Aston Martin,8.0,15.0
3,2023,1,Ferrari,12.0,23.0
18,2023,1,Ferrari,0.0,12.0


In [6]:
# Sort by driver-season-round for rolling stats
df = df.sort_values(by=['Driver.code', 'season', 'round'])
driver_groups = df.groupby('Driver.code')

# Average finishing position in last 5 races (before this race)
df['driver_avg_last5'] = (
    driver_groups['position']
      .rolling(window=5, min_periods=1)
      .mean()
      .shift(1)
      .reset_index(level=0, drop=True)
)

# Number of podiums in last 5 races
df['driver_podiums_last5'] = (
    driver_groups['podium']
      .rolling(window=5, min_periods=1)
      .sum()
      .shift(1)
      .reset_index(level=0, drop=True)
)

# Best (lowest) finishing position in last 5 races
df['driver_best_last5'] = (
    driver_groups['position']
      .rolling(window=5, min_periods=1)
      .min()
      .shift(1)
      .reset_index(level=0, drop=True)
)

# Worst (highest) finishing position in last 5 races
df['driver_worst_last5'] = (
    driver_groups['position']
      .rolling(window=5, min_periods=1)
      .max()
      .shift(1)
      .reset_index(level=0, drop=True)
)

df[['Driver.code', 'season', 'round',
    'position', 'podium',
    'driver_avg_last5', 'driver_podiums_last5',
    'driver_best_last5', 'driver_worst_last5']].head(15)

,Driver.code,season,round,position,podium,driver_avg_last5,driver_podiums_last5,driver_best_last5,driver_worst_last5
9,ALB,2023,1,10,0,NaN,NaN,NaN,NaN
38,ALB,2023,2,19,0,10.000000,0.0,10.0,10.0
58,ALB,2023,3,19,0,14.500000,0.0,10.0,19.0
71,ALB,2023,4,12,0,16.000000,0.0,10.0,19.0
93,ALB,2023,5,14,0,15.000000,0.0,10.0,19.0
2,ALO,2023,1,3,1,14.800000,0.0,10.0,19.0
22,ALO,2023,2,3,1,3.000000,1.0,3.0,3.0
42,ALO,2023,3,3,1,3.000000,2.0,3.0,3.0
63,ALO,2023,4,4,0,3.000000,3.0,3.0,3.0
82,ALO,2023,5,3,1,3.250000,3.0,3.0,4.0


In [7]:
# Sort to make teammate pairing consistent
df = df.sort_values(by=['season', 'round', 'Constructor.name', 'grid'])

group_cols = ['season', 'round', 'Constructor.name']

# Teammate grid: reverse each team group so each driver gets the other's grid
df['teammate_grid'] = (
    df.groupby(group_cols)['grid']
      .transform(lambda x: x.iloc[::-1].values)
)

# Teammate final position
df['teammate_position'] = (
    df.groupby(group_cols)['position']
      .transform(lambda x: x.iloc[::-1].values)
)

# Deltas vs teammate
df['grid_delta_vs_teammate'] = df['grid'] - df['teammate_grid']
df['position_delta_vs_teammate'] = df['position'] - df['teammate_position']

df[['season', 'round', 'Constructor.name', 'Driver.code',
    'grid', 'teammate_grid', 'grid_delta_vs_teammate',
    'position', 'teammate_position', 'position_delta_vs_teammate']].head(20)

,season,round,Constructor.name,Driver.code,grid,teammate_grid,grid_delta_vs_teammate,position,teammate_position,position_delta_vs_teammate
7,2023,1,Alfa Romeo,BOT,12,13,-1,8,16,-8
15,2023,1,Alfa Romeo,ZHO,13,12,1,16,8,8
10,2023,1,AlphaTauri,TSU,14,19,-5,11,14,-3
13,2023,1,AlphaTauri,DEV,19,14,5,14,11,3
17,2023,1,Alpine F1 Team,OCO,9,20,-11,18,9,9
8,2023,1,Alpine F1 Team,GAS,20,9,11,9,18,-9
2,2023,1,Aston Martin,ALO,5,8,-3,3,6,-3
5,2023,1,Aston Martin,STR,8,5,3,6,3,3
18,2023,1,Ferrari,LEC,3,4,-1,19,4,15
3,2023,1,Ferrari,SAI,4,3,1,4,19,-15


In [8]:
if 'Circuit.circuitId' in df.columns:
    df['circuit_id'] = df['Circuit.circuitId']

    street_tracks = [
        'monaco', 'baku', 'singapore', 'las_vegas', 'miami', 'jeddah'
    ]

    counterclockwise_tracks = [
        'interlagos', 'cota', 'yas_marina', 'istanbul'
    ]

    df['is_street_track'] = df['circuit_id'].isin(street_tracks).astype(int)
    df['is_ccw'] = df['circuit_id'].isin(counterclockwise_tracks).astype(int)

    # Compute "grid advantage" per circuit
    grid_advantage = (
        df.groupby('circuit_id')
          .apply(lambda x: (x['grid'] - x['position']).mean())
          .rename('grid_advantage')
          .reset_index()
    )

    df = df.merge(grid_advantage, on='circuit_id', how='left')
else:
    print("No Circuit.circuitId column found; skipping circuit features for now.")

No Circuit.circuitId column found; skipping circuit features for now.


In [9]:
df.to_csv("../data/f1_2023_engineered.csv", index=False)

df[['Driver.code', 'Constructor.name', 'grid', 'position', 'podium',
    'constructor_cum_points',
    'driver_avg_last5', 'driver_podiums_last5',
    'grid_delta_vs_teammate', 'position_delta_vs_teammate']].head()

,Driver.code,Constructor.name,grid,position,podium,constructor_cum_points,driver_avg_last5,driver_podiums_last5,grid_delta_vs_teammate,position_delta_vs_teammate
7,BOT,Alfa Romeo,12,8,0,0.0,3.2,4.0,-1,-8
15,ZHO,Alfa Romeo,13,16,0,4.0,1.4,5.0,1,8
10,TSU,AlphaTauri,14,11,0,4.0,9.8,0.0,-5,-3
13,DEV,AlphaTauri,19,14,0,0.0,13.6,0.0,5,3
17,OCO,Alpine F1 Team,9,18,0,2.0,13.2,0.0,-11,9
